In [1]:
# script to convert *.las files to potree 1.6 compatible octrees using laspy and numpy.
# A great source for *laz files is http://3dsm.bk.tudelft.nl/matahn 
#
# Code is setup to enable parallel processing
# TODO: optimize code
# TODO: allow multicore processing
# TODO: allow more paramaters than just x,y,z to be parsed
# TODO: change code to module
# TODO: 

In [2]:
import os
import laspy 
import numpy as np
from pprint import pprint
import json
from pathlib import Path
import copy

In [3]:
# convert laz to las using las2las
LAS_DIRECTORY = 'F:\pointclouds'
p = Path(LAS_DIRECTORY)
for f in p.glob('*.laz'):
    fn = f.__str__()
    fn_new = fn.replace('.laz', '.las')
    if not os.path.exists(fn_new):
        cmd = r'"C:\Program Files (x86)\lastools\bin\las2las.exe" -i {} -o {} '.format(fn,fn_new)
        os.system(cmd)

In [4]:
# make a generator for file headers
# http://pythonhosted.org/laspy/header.html
p = Path(LAS_DIRECTORY)
def headers():
    for f in p.glob('*.las'):
        fn = f.__str__()
        lasfile = laspy.file.File(fn)
        yield lasfile.header    

In [5]:
bounds = np.array((
    np.max([hdr.max for hdr in headers()],axis=0),
    np.min([hdr.min for hdr in headers()],axis=0),
    ))
print(bounds)
[print(hdr.scale) for hdr in headers()]

[[  1.80816630e+05   4.89604710e+05   1.67080000e+02]
 [  7.09043200e+04   3.16084240e+05  -1.98100000e+01]]
[0.01, 0.01, 0.01]
[0.01, 0.01, 0.01]
[0.01, 0.01, 0.01]
[0.01, 0.01, 0.01]
[0.01, 0.01, 0.01]
[0.01, 0.01, 0.01]
[0.01, 0.01, 0.01]
[0.01, 0.01, 0.01]


[None, None, None, None, None, None, None, None]

In [6]:
# define maximum bounding cube for data. MUST be cube, so all sides equal length
min_size = max(bounds[0,:] - bounds[1,:])
print(min_size)
suggested_centers = ((bounds[0,:] + bounds[1,:])/2).astype(np.dtype('<i4'))
suggested_levels = np.ceil(np.log2(min_size/100))
print(suggested_levels,suggested_centers)

173520.47
11.0 [125860 402844     73]


In [7]:
CENTERS = {'x': 125860, 'y':402844, 'z':70000}
SMALLEST_TILE_SIZE = 100 # meter
LEVELS = 11 
SCALE = 0.01
OUTPUT_DIR = Path(r'D:\pointclouds\data')
# define size as nice power of two (not necessary)
SIZE = SMALLEST_TILE_SIZE * 2 ** LEVELS
BBOX = {k: np.array([v-SIZE/2, v+SIZE/2]).astype('i4') for k, v in CENTERS.items()}

print(BBOX)
# check if all points are within bounding box
print([[h_min>=BBOX[dim][0] and h_max<=BBOX[dim][1] for
       dim, h_min, h_max in zip(['x','y','z'],
                                hdr.min,
                                hdr.max)] for
       hdr in headers()])



{'y': array([300444, 505244]), 'z': array([-32400, 172400]), 'x': array([ 23460, 228260])}
[[True, True, True], [True, True, True], [True, True, True], [True, True, True], [True, True, True], [True, True, True], [True, True, True], [True, True, True]]


In [8]:
HIERACHY_STEP_SIZE = 4
def address_to_filename(address):
    if address is '':
        return Path(OUTPUT_DIR,'r','r.bin')

    parts = [address[i:i+HIERACHY_STEP_SIZE] for i in range(0, len(address)+1, HIERACHY_STEP_SIZE)]
    parts[-1]='r' + address + '.bin'
    return Path(OUTPUT_DIR,'r',*parts).__str__()

print(address_to_filename('000'))
print(address_to_filename('112312312'))  
print(address_to_filename('7000'))
print(address_to_filename(''))


D:\pointclouds\data\r\r000.bin
D:\pointclouds\data\r\1123\1231\r112312312.bin
D:\pointclouds\data\r\7000\r7000.bin
D:\pointclouds\data\r\r.bin


In [9]:
def address_2_origin(address):
    """ Each tile is saved in coordinates relitive to the origin of the tile.
    The origin of a file can be calculated from the bounding box (BBOX) and the address
    """
    if address == '':
        address = '0'

    # return origin relative to bounding box origin in unscaled values
    level = len(address)
    bin_parts = [bin(int(c)+8)[3::] for c in address]
    x0 = int((int(''.join([b[0] for b in bin_parts]),2) / 2**level) * SIZE / SCALE)
    y0 = int((int(''.join([b[1] for b in bin_parts]),2) / 2**level) * SIZE / SCALE)
    z0 = int((int(''.join([b[2] for b in bin_parts]),2) / 2**level) * SIZE / SCALE)
    return (x0, y0, z0)

print(*address_2_origin('000'))
print(*address_2_origin('112312312'))  
print(*address_2_origin('7000'))
print(*address_2_origin(''))

0 0 0
0 4360000 17520000
10240000 10240000 10240000
0 0 0


In [10]:
p = Path(OUTPUT_DIR,'r')
for n in range(0,LEVELS):
    print('removed',len([f.unlink() for f in p.rglob('r'+ ('?' * n) + '.bin')]),'files at level', n)

removed 0 files at level 0
removed 0 files at level 1
removed 0 files at level 2
removed 0 files at level 3
removed 0 files at level 4
removed 0 files at level 5
removed 0 files at level 6
removed 0 files at level 7
removed 0 files at level 8
removed 0 files at level 9
removed 0 files at level 10


In [11]:
p = Path(LAS_DIRECTORY)
def lasfiles():
    for f in p.glob('*.las'):
         yield laspy.file.File(f.__str__())

In [12]:
#TODO: split in subfunctions

for lasfile in lasfiles():
    
    hdr = lasfile.header
    print(hdr)
    # to avoid duplicating the point coordinates in memory, discretize the points based on their raw coordinates
    # in unscaled integers
    
    bbox_unscaled = {dim: ((BBOX[dim]-offset)/scale).astype('i4') for 
                   dim, scale, offset in zip(['x','y','z'], hdr.scale, hdr.offset)}

    # determine the bins to classify points
    bins = {dim: np.linspace(bbox_unscaled[dim][0],
                             bbox_unscaled[dim][1],
                             2**(LEVELS)+1
                       ).astype('i4')[1:-1] for dim in ['x','y','z']}

    # put all points in bins
    binned = [np.digitize(getattr(lasfile,dim.upper()),bins[dim]) for dim in ['x','y','z']]

    for x in set(binned[0]):
        p = np.equal(binned[0],x)
        subset_x = [b[p] for b in binned]
        address_x = bin(x + 2**LEVELS)[-LEVELS::]

        for y in set(subset_x[1]):
            q = np.equal(subset_x[1],y)
            subset_xy = [b[q] for b in subset_x]
            address_y = bin(y + 2**LEVELS)[-LEVELS::]

            for z in set(subset_xy[2]):
                r = np.equal(subset_xy[2],z)
                subset_xyz = [b[r] for b in subset_xy]
                address_z = bin(z + 2**LEVELS)[-LEVELS::]

                # make filename of 0-7's
                # https://github.com/potree/potree/blob/master/docs/file_format.md
                address = ''.join([str(int(''.join((x,y,z)),2)) for x,y,z in zip(address_x,address_y,address_z)])
                filename = address_to_filename(address)
                # print(filename)

                # extract relevant subset of data
                subset = np.where(p)[0][q][r]

                # convert points from int32 relative to hdr.scale and hdr.offset to uint32 relative to SCALE, BBOX
                # and the tile box address
                x0, y0, z0 = address_2_origin(address)

                if not os.path.exists(os.path.dirname(filename)):
                    os.makedirs(os.path.dirname(filename))
                np.vstack((
                     (((lasfile.X[subset] * hdr.scale[0]) + hdr.offset[0] - BBOX['x'][0]) / SCALE).astype('<u4') - x0,
                     (((lasfile.Y[subset] * hdr.scale[1]) + hdr.offset[1] - BBOX['y'][0]) / SCALE).astype('<u4') - y0,
                     (((lasfile.Z[subset] * hdr.scale[2]) + hdr.offset[2] - BBOX['z'][0]) / SCALE).astype('<u4') - z0
                     )).transpose().tofile(filename)
    lasfile.close()


MemoryError: 

[]

In [11]:
# from pathlib import Path
import re
only_digits = re.compile(r"\D")

p = Path(OUTPUT_DIR,'r')
addresses = [only_digits.sub("",paths.parts[-1]) for paths in p.glob('**/r*.bin')]
print(addresses[::100])

['24343422457', '24343426633', '24343604102', '26161020166', '26161022506', '26161024540', '26161026473', '26161060162', '26161062035', '26161062720', '26161064560', '26161066413', '26161200522', '26161204360', '26161206724', '26161240675', '26161244073', '26161244706', '26161246677', '26161420340', '26161422362', '26161600415', '26161602524', '26741666326', '26743440706', '26743444142', '26743446011', '26743446651', '26743462651', '26743464522', '26743466413', '26743644162', '26745222613', '26745226637', '26745262722', '26745266746', '26745626035', '26747000251', '26747002106', '26747002744', '26747004611', '26747006431', '26747020277', '26747022144', '26747024073', '26747024706', '26747026617', '26747040455', '26747042342', '26747044213', '26747046071', '26747046704', '26747060564', '26747062455', '26747064344', '26747066257', '26747200453', '26747240142', '26747400075', '26747400722', '26747402633', '26747404706', '26747420106', '26747422031', '26747424055', '26747426344', '26747604

In [19]:
# convert tree to tree
tree = {}
for a in addresses:
    t = tree
    for s in a:
        if s not in t:
            t[s] = {} 
        t = t[s]

In [20]:
# print(json.dumps(tree,sort_keys=True, indent=2, separators=(',', ': ')))
# convert to list of all possible tiles, and their children

def nodes_to_list(prefix, key, value):
    hrc = [{'address':prefix+key,
            'children': tuple(value.keys())}, 
          ]
    
    [hrc.extend(nodes_to_list(prefix+key,k,v)) for k, v in value.items()]
    
    return hrc

hrc = nodes_to_list('','',tree)
hrc.sort(key=lambda k: (len(k['address']), k['address']))


In [33]:
p = Path(r'D:\pointclouds\data\r\r.bin')
print(p)
print(p.parent)

D:\pointclouds\data\r\r.bin
D:\pointclouds\data\r


In [34]:
for item in hrc:
    filename = address_to_filename(item['address'])
    item.update(
        {
            'mask': sum([2 ** int(c) for c in item['children']]),
            'filename': filename,
            'directory': Path(filename).parent,
            'parentDirectory': Path(address_to_filename(item['address'][:-1])).parent
        })
    
pprint(hrc[-5:-1])

[{'address': '26747604506',
  'children': (),
  'directory': WindowsPath('D:/pointclouds/data/r/2674/7604'),
  'filename': 'D:\\pointclouds\\data\\r\\2674\\7604\\r26747604506.bin',
  'mask': 0,
  'parentDirectory': WindowsPath('D:/pointclouds/data/r/2674/7604')},
 {'address': '26747604520',
  'children': (),
  'directory': WindowsPath('D:/pointclouds/data/r/2674/7604'),
  'filename': 'D:\\pointclouds\\data\\r\\2674\\7604\\r26747604520.bin',
  'mask': 0,
  'parentDirectory': WindowsPath('D:/pointclouds/data/r/2674/7604')},
 {'address': '26747604522',
  'children': (),
  'directory': WindowsPath('D:/pointclouds/data/r/2674/7604'),
  'filename': 'D:\\pointclouds\\data\\r\\2674\\7604\\r26747604522.bin',
  'mask': 0,
  'parentDirectory': WindowsPath('D:/pointclouds/data/r/2674/7604')},
 {'address': '26747604524',
  'children': (),
  'directory': WindowsPath('D:/pointclouds/data/r/2674/7604'),
  'filename': 'D:\\pointclouds\\data\\r\\2674\\7604\\r26747604524.bin',
  'mask': 0,
  'parentDirec

In [35]:
dt = np.dtype([('x','<u4'),('y','<u4'),('z','<u4')])
def read_file_at_address(address):
    points = np.fromfile(address_to_filename(address),dtype=dt)
    x0, y0, z0 = address_2_origin(address)
    points['x'] += x0
    points['y'] += y0
    points['z'] += z0
    return(points)

In [37]:
for h in hrc[::-1]:
    if not os.path.exists(h['filename']):
        points = np.hstack([read_file_at_address(h['address']+child) for child in h['children']])
        
        x0, y0, z0 = address_2_origin(h['address'])
        points['x'] -= x0
        points['y'] -= y0
        points['z'] -= z0

        points[::8].tofile(h['filename'])
    h['n_points'] = int(os.stat(h['filename']).st_size / dt.itemsize)
        

TypeError: argument should be string, bytes or integer, not WindowsPath

In [17]:
hrc_dtype = np.dtype([
    ("mask", "<u1"),            
    ("n_points", "<u4"),   
    ])

import re
only_digits = re.compile(r"\D")


for d in set(h['directory'] for h in hrc):
    hrc_data = np.array(
        [(h['mask'],h['n_points']) for h in hrc if 
         h['parentDirectory'] == d or h['directory'] == d],
        dtype=hrc_dtype)

    hrc_data.tofile(os.path.join(d, 'r' + only_digits.sub("",d) + '.hrc'))



KeyError: 'directory'

In [18]:
options = {
    "version": "1.6",
    "octreeDir": OUTPUT_DIR,
    "boundingBox": {
        "lx": BBOX['x'][0],
        "ly": BBOX['y'][0],
        "lz": BBOX['z'][0],
        "ux": BBOX['x'][1],
        "uy": BBOX['y'][1],
        "uz": BBOX['z'][1]
    },
    "tightBoundingBox": {
        "lx": hdr.min[0],
        "ly": hdr.min[1],
        "lz": hdr.min[2],
        "ux": hdr.max[0],
        "uy": hdr.max[1],
        "uz": hdr.max[2]
    },
    "pointAttributes": ["POSITION_CARTESIAN"],
    "spacing": 0.01,
    "scale": SCALE,
    "hierarchyStepSize": HIERACHY_STEP_SIZE,
}
print(json.dumps(options,sort_keys=True, indent=2, separators=(',', ': ')))
with open('cloud.js','w') as f:
    json.dump(options,f)

NameError: name 'hdr' is not defined